In [1]:
import pandas as pd
import numpy as np
import os

In [2]:
import pandas as pd
import glob

path = '' # use your path
all_files = glob.glob(path.join('../data/all_data/*.csv'))

li = []

for filename in all_files:
    df = pd.read_csv(filename, index_col=None, header=0)
    li.append(df)

In [3]:
frame = pd.concat(li, axis=0, ignore_index=True)

In [4]:
pd.to_datetime(frame['created_at'], unit='s').sort_values(ascending=False)

12346   2022-04-21 00:16:03
28926   2022-04-20 21:37:58
216     2022-04-20 21:37:58
800     2022-04-20 11:18:04
9015    2022-04-20 09:57:16
                ...        
30567   2009-04-29 19:52:44
12308   2009-04-28 22:14:23
27307   2009-04-27 04:44:17
30568   2009-04-25 05:37:50
27245   2009-04-23 03:06:59
Name: created_at, Length: 30992, dtype: datetime64[ns]

In [5]:
frame.to_csv('master_df.csv')